In [ ]:
import pandas as pd 
import numpy as np 

import re

import matplotlib.pyplot as plt
import seaborn as sns 




In [66]:
#Importing data 
df= pd.read_csv("C:/Users/ADMIN\Documents/My projects/Creator-Brand-Recommender/combined_instagram_data.csv")

#Viewing first few rows of the data
df.head()

,account,fbid,id,followers,posts_count,is_business_account,is_professional_account,is_verified,avg_engagement,external_url,...,partner_id,business_address,timestamp,input,error,error_code,warning,warning_code,source_file,processed_at
0,iamkevinedson,1.784141e+16,7.128404e+09,578.0,171.0,0.0,0.0,0.0,0.0210,NaN,...,7.128404e+09,NaN,2024-12-15T09:46:14.094Z,"{""url"":""https://www.instagram.com/iamkevinedso...",NaN,NaN,NaN,NaN,https___www.instagram.com_iamkevinedson__s_m4p...,2024-12-15 16:33:32
1,flygirl_koki,1.784145e+16,4.827237e+10,29581.0,142.0,0.0,0.0,1.0,0.0411,NaN,...,4.827237e+10,NaN,2024-12-15T10:23:25.422Z,"{""url"":""https://www.instagram.com/flygirl_koki/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_flygirl_koki__s_m4pg...,2024-12-15 16:33:32
2,_lanoi,1.784140e+16,2.338770e+08,1047.0,89.0,0.0,0.0,0.0,NaN,NaN,...,2.338770e+08,NaN,2024-12-15T10:10:07.013Z,"{""url"":""https://www.instagram.com/_lanoi/""}",NaN,NaN,NaN,NaN,https___www.instagram.com__lanoi__s_m4pg2xkw1k...,2024-12-15 16:33:33
3,darassacmg255,1.784140e+16,5.591099e+08,1637207.0,17.0,1.0,1.0,1.0,0.0062,NaN,...,5.591099e+08,NaN,2024-12-15T09:51:46.801Z,"{""url"":""https://www.instagram.com/darassacmg25...",NaN,NaN,NaN,NaN,https___www.instagram.com_darassacmg255__s_m4p...,2024-12-15 16:33:33
4,newhavenrestaurant,1.784145e+16,5.305925e+10,1923.0,263.0,1.0,1.0,0.0,0.0053,"[""https://www.facebook.com/profile.php?id=1129...",...,5.305925e+10,"Kilimani , likoni close off denis pritt road, ...",2024-12-15T10:12:37.886Z,"{""url"":""https://www.instagram.com/newhavenrest...",NaN,NaN,NaN,NaN,https___www.instagram.com_newhavenrestaurant__...,2024-12-15 16:33:33


In [67]:
#viewing last rows of the data
df.tail()

,account,fbid,id,followers,posts_count,is_business_account,is_professional_account,is_verified,avg_engagement,external_url,...,partner_id,business_address,timestamp,input,error,error_code,warning,warning_code,source_file,processed_at
2527,ledradmedia,1.784142e+16,1.713318e+10,1168.0,429.0,1.0,1.0,0.0,0.0022,"[""http://www.ledrad.com/""]",...,1.713318e+10,NaN,2024-12-15T09:49:11.456Z,"{""url"":""https://www.instagram.com/ledradmedia/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_ledradmedia__s_m4pfb...,2024-12-15 16:33:50
2528,salimesta_,1.784144e+16,3.591003e+10,9535.0,203.0,1.0,1.0,0.0,0.1877,"[""https://youtu.be/6tMD2RPKImk?si=fDcgpoRLPmjx...",...,3.591003e+10,NaN,2024-12-15T10:05:28.016Z,"{""url"":""https://www.instagram.com/salimesta_/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_salimesta___s_m4pfwy...,2024-12-15 16:33:50
2529,smil3y.gee,1.784142e+16,2.103978e+10,536.0,5.0,0.0,1.0,0.0,0.5126,NaN,...,2.103978e+10,NaN,2024-12-15T09:45:14.464Z,"{""url"":""https://www.instagram.com/smil3y.gee/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_smil3y.gee__s_m4pf6x...,2024-12-15 16:33:50
2530,ngaira_brian,1.784140e+16,1.427502e+09,3300.0,459.0,0.0,1.0,0.0,0.0569,"[""https://linktr.ee/ngairalidonde"",""https://yo...",...,1.427502e+09,NaN,2024-12-15T10:36:10.685Z,"{""url"":""https://www.instagram.com/ngaira_brian/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_ngaira_brian__s_m4ph...,2024-12-15 16:33:50
2531,fridah_mumbe,1.784140e+16,1.826805e+09,7690.0,86.0,0.0,1.0,0.0,0.0733,"[""http://magicvibeawards.com/""]",...,1.826805e+09,NaN,2024-12-15T10:42:41.703Z,"{""url"":""https://www.instagram.com/fridah_mumbe/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_fridah_mumbe__s_m4ph...,2024-12-15 16:33:50


In [68]:
#Getting list of columns in the data
df.columns

Index(['account', 'fbid', 'id', 'followers', 'posts_count',
       'is_business_account', 'is_professional_account', 'is_verified',
       'avg_engagement', 'external_url', 'biography', 'business_category_name',
       'category_name', 'post_hashtags', 'following', 'posts',
       'profile_image_link', 'profile_url', 'profile_name', 'highlights_count',
       'highlights', 'full_name', 'is_private', 'bio_hashtags',
       'business_email', 'phone_number', 'url', 'is_joined_recently',
       'has_channel', 'partner_id', 'business_address', 'timestamp', 'input',
       'error', 'error_code', 'warning', 'warning_code', 'source_file',
       'processed_at'],
      dtype='object')

In [69]:
#Shape of the data
print (f'Number of entries: {df.shape[0]}')
print (f'Number of columns: {df.shape[1]}')

Number of entries: 2532
Number of columns: 39


In [ ]:


# Preprocess the biography text: lowercasing, removing punctuation, and tokenizing
def clean_and_tokenize(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.split()  # Tokenize the text into words
    return words

# Apply the function to the biography column
all_words = df['biography'].fillna('').apply(clean_and_tokenize)

#Make a siingle list and get unique words
unique_keywords = set([word for sublist in all_words for word in sublist])

# Display the unique keywords
print(unique_keywords)


{'standup', '214k', 'animator', 'sparkscorporates', 'creation', '𝒜𝑔𝑒𝓃𝓉', 'javaphile', 'msa', 'selectmodelstockholm', 'experience', 'lit', 'الجبيله', 'jauloma', 'kie_kie_tv_', '𝚌𝚘𝚗𝚌𝚎𝚙𝚝', 'de', 'media', 'meaningful', 'uselemfi', 'awardee', 'christ', 'academies', 'bornliving', 'kitakaswangzavenuecom', 'seated', 'destined', 'ability', 'peluminubinetworkcom', 'mukbangs', 'الدنيا', 'highquality', 'subscribers', 'תבדקו', '9th', 'unlearning', 'إني', 'museum', 'coaching', 'presentertheweddingshow', 'swag', 'mama', 'text', 'well', 'yardenetinsrecordscom', 'shirkomedia_ent', 'fine', 'fromoutersouth', 'britaniafoods', 'c', 'billboardcharts', 'pearl', 'desserts', 'ambivert', 'bookingkarungmailcom', 'hannahjaynegrailtalentcom', 'nurturing', '𝐴𝑙', '5676127', 'programmes', 'pallonji', 'woodways', 'touchbybetty', 'aroap', 'naivas', 'ditalia', 'vessel', 'jimseuhh', 'zerowaste', 'child', 'founderartistic', 'hausofafro', 'vocalist', 'rapper', 'yankees', 'changed', '780', 'jimseuhgiseleparisfr', 'making', 

In [ ]:
# Define keyword dictionaries for niches
niche_keywords = {
    'Health': ['health','fitness', 'workout', 'gym', 'health', 'trainer', 'yoga','fit','dermatologist','swimmer','pharmacy','hospital','clinic','surgery','wellness'],
    'Lifestyle':['lifestyle','mum','dad','mother','father','mama','marriage', 'married','therapy','spa','pet','model','wife','parent'],
    'Fashion': ['fashion', 'style', 'designer', 'clothing', 'beauty', 'makeup','clothes','dressing','wear','lotion','perfume','skin care','stylist'],
    'Travel': ['travel', 'wanderlust', 'vacation', 'adventure', 'explore','safari'],
    'Tech': ['tech', 'gadgets', 'software', 'developer', 'AI', 'programmer','machine','data'],
    'Food': ['food', 'recipe', 'chef', 'cooking', 'baking', 'restaurant','hotel','kitchen','foodie'],
    'Art': ['art', 'artist', 'drawing', 'painting', 'illustration', 'design','comedy','podcast','craft','photograph','choreography','actor','actress','writer','host','poet','storyteller','author'],
    'Music': ['music', 'singer', 'band', 'producer', 'songwriter', 'DJ','rapper','vocalist','saxophonist','musician'],
    'Business':['business','finance','financial','money','consultancy','shop','consulting','entrepreneur','store'],
}

# Preprocess the biography text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df['biography_clean'] = df['biography'].fillna('').apply(clean_text)

# Match keywords to assign a niche
def assign_niche(bio, keywords_dict):
    for niche, keywords in keywords_dict.items():
        if any(keyword in bio for keyword in keywords):
            return niche
    return 'Other'  # Default category for unmatched biographies

df['niche'] = df['biography_clean'].apply(lambda x: assign_niche(x, niche_keywords))

# Save the updated dataset 
save_path = "C:/Users/ADMIN/Documents/My projects/Creator-Brand-Recommender/combined_instagram_data_with_niches.csv"
df.to_csv(save_path, index=False)

print(f"Dataset with niches saved successfully at: {save_path}")


Dataset with niches saved successfully at: C:/Users/ADMIN/Documents/My projects/Creator-Brand-Recommender/combined_instagram_data_with_niches.csv


In [61]:
data = pd.read_csv("C:/Users/ADMIN/Documents/My projects/Creator-Brand-Recommender/combined_instagram_data_with_niches.csv")
data.head()

,account,fbid,id,followers,posts_count,is_business_account,is_professional_account,is_verified,avg_engagement,external_url,...,timestamp,input,error,error_code,warning,warning_code,source_file,processed_at,biography_clean,niche
0,iamkevinedson,1.784141e+16,7.128404e+09,578.0,171.0,0.0,0.0,0.0,0.0210,NaN,...,2024-12-15T09:46:14.094Z,"{""url"":""https://www.instagram.com/iamkevinedso...",NaN,NaN,NaN,NaN,https___www.instagram.com_iamkevinedson__s_m4p...,2024-12-15 16:33:32,\n entrepreneur\n super dad\nadrenalin junky\n...,Lifestyle
1,flygirl_koki,1.784145e+16,4.827237e+10,29581.0,142.0,0.0,0.0,1.0,0.0411,NaN,...,2024-12-15T10:23:25.422Z,"{""url"":""https://www.instagram.com/flygirl_koki/""}",NaN,NaN,NaN,NaN,https___www.instagram.com_flygirl_koki__s_m4pg...,2024-12-15 16:33:32,first african b787 dreamliner female captain ...,Fashion
2,_lanoi,1.784140e+16,2.338770e+08,1047.0,89.0,0.0,0.0,0.0,NaN,NaN,...,2024-12-15T10:10:07.013Z,"{""url"":""https://www.instagram.com/_lanoi/""}",NaN,NaN,NaN,NaN,https___www.instagram.com__lanoi__s_m4pg2xkw1k...,2024-12-15 16:33:33,protect your energy,Other
3,darassacmg255,1.784140e+16,5.591099e+08,1637207.0,17.0,1.0,1.0,1.0,0.0062,NaN,...,2024-12-15T09:51:46.801Z,"{""url"":""https://www.instagram.com/darassacmg25...",NaN,NaN,NaN,NaN,https___www.instagram.com_darassacmg255__s_m4p...,2024-12-15 16:33:33,NaN,Other
4,newhavenrestaurant,1.784145e+16,5.305925e+10,1923.0,263.0,1.0,1.0,0.0,0.0053,"[""https://www.facebook.com/profile.php?id=1129...",...,2024-12-15T10:12:37.886Z,"{""url"":""https://www.instagram.com/newhavenrest...",NaN,NaN,NaN,NaN,https___www.instagram.com_newhavenrestaurant__...,2024-12-15 16:33:33,your plug for tasty kienyeji african dishes we...,Other


In [62]:
niche_counts = df['niche'].value_counts()
print("Count of each unique niche:")
print(niche_counts)



Count of each unique niche:
niche
Other        1295
Art           375
Lifestyle     254
Music         139
Health        124
Business       91
Fashion        89
Food           83
Travel         54
Tech           28
Name: count, dtype: int64


In [72]:
data.describe()

,fbid,id,followers,posts_count,is_business_account,is_professional_account,is_verified,avg_engagement,post_hashtags,following,highlights_count,highlights,is_private,business_email,phone_number,is_joined_recently,has_channel,partner_id,error,error_code
count,2.531000e+03,2.531000e+03,2.531000e+03,2531.000000,2531.000000,2531.000000,2531.000000,2352.000000,0.0,2531.000000,2531.000000,0.0,2531.000000,0.0,0.0,2531.0,2531.0,2.531000e+03,0.0,0.0
mean,1.784141e+16,1.083244e+10,3.406060e+06,1747.451205,0.337416,0.836823,0.407744,0.077982,NaN,1443.101146,16.711181,NaN,0.084552,NaN,NaN,0.0,0.0,1.083244e+10,NaN,NaN
std,1.797575e+10,1.804196e+10,2.598881e+07,6908.049407,0.472922,0.369600,0.491512,0.330275,NaN,1636.081122,284.062631,NaN,0.278268,NaN,NaN,0.0,0.0,1.804196e+10,NaN,NaN
min,1.784140e+16,9.556100e+04,3.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000100,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.0,0.0,9.556100e+04,NaN,NaN
25%,1.784140e+16,4.735331e+08,3.403500e+03,89.000000,0.000000,1.000000,0.000000,0.006600,NaN,316.500000,1.000000,NaN,0.000000,NaN,NaN,0.0,0.0,4.735331e+08,NaN,NaN
50%,1.784140e+16,2.031324e+09,2.908300e+04,362.000000,0.000000,1.000000,0.000000,0.023250,NaN,920.000000,5.000000,NaN,0.000000,NaN,NaN,0.0,0.0,2.031324e+09,NaN,NaN
75%,1.784141e+16,8.180294e+09,4.750705e+05,1209.000000,1.000000,1.000000,1.000000,0.063325,NaN,1899.000000,11.000000,NaN,0.000000,NaN,NaN,0.0,0.0,8.180294e+09,NaN,NaN
max,1.784147e+16,7.025228e+10,6.829453e+08,159217.000000,1.000000,1.000000,1.000000,12.187200,NaN,7505.000000,14229.000000,NaN,1.000000,NaN,NaN,0.0,0.0,7.025228e+10,NaN,NaN


In [73]:
#checking for missing  values
data.isnull().sum()

account                       1
fbid                          1
id                            1
followers                     1
posts_count                   1
is_business_account           1
is_professional_account       1
is_verified                   1
avg_engagement              180
external_url                758
biography                   196
business_category_name     2455
category_name               563
post_hashtags              2532
following                     1
posts                       176
profile_image_link            1
profile_url                   1
profile_name                 46
highlights_count              1
highlights                 2532
full_name                    46
is_private                    1
bio_hashtags               2276
business_email             2532
phone_number               2532
url                           1
is_joined_recently            1
has_channel                   1
partner_id                    1
business_address           2421
timestam

In [74]:
#Checking for duplicates
data.duplicated().sum()

0

In [75]:
#Checked for errors, and data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532 entries, 0 to 2531
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   account                  2531 non-null   object 
 1   fbid                     2531 non-null   float64
 2   id                       2531 non-null   float64
 3   followers                2531 non-null   float64
 4   posts_count              2531 non-null   float64
 5   is_business_account      2531 non-null   float64
 6   is_professional_account  2531 non-null   float64
 7   is_verified              2531 non-null   float64
 8   avg_engagement           2352 non-null   float64
 9   external_url             1774 non-null   object 
 10  biography                2336 non-null   object 
 11  business_category_name   77 non-null     object 
 12  category_name            1969 non-null   object 
 13  post_hashtags            0 non-null      float64
 14  following               